In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium

In [23]:
# campout locations
places_link = 'https://docs.google.com/spreadsheets/d/1lGYzZ3xcjmLJRE_3w2uths773rmiLo--3D0ot1lMP9U/export?format=csv'
places = pd.read_csv(places_link)

# links to google photos
photos_link = 'https://docs.google.com/spreadsheets/d/1zbNJclqyhI2PlKiUXJcWzufQfhJTIWXcZx2Iqud76P4/export?format=csv'
photo_links = pd.read_csv(photos_link)


places = pd.merge(places, photo_links, on='id', how='outer')

print('Number of places: {}\nNumber of trips: {}'.format(len(places.name_x.unique()), len(places)))

places.head()

Number of places: 27
Number of trips: 40


,id,name_x,lon,lat,original_trip,name_y,year,link
0,1,Turkey Run,-87.2006,39.8909,NaN,NaN,NaN,NaN
1,2,Spring Mill,-86.4168,38.7336,NaN,Spring Mill,1987.0,https://photos.app.goo.gl/qyyLSwt9HAU5gmXS9
2,2,Spring Mill,-86.4168,38.7336,NaN,Spring Mill,2000.0,https://photos.app.goo.gl/Vx8UBSkE6rN66ANF7
3,2,Spring Mill,-86.4168,38.7336,NaN,Spring Mill,2002.0,https://photos.app.goo.gl/bJXDsZBp4Usuq45m9
4,2,Spring Mill,-86.4168,38.7336,NaN,Spring Mill,2000.0,https://photos.app.goo.gl/Vx8UBSkE6rN66ANF7


In [24]:

m = folium.Map(location=[39, -95], zoom_start=4.4, tiles='Stamen Terrain')


# function to create n links in the marker popup when there are multiple years at one location
def get_popup_links():
    
    name = row['name_x']
    html_str = '<a href="{}"target="_blank"> {:.0f} </a>\n<br>'
    n_link = places[places.name_x==row['name_x']].link.count()
    
    links = places[places.name_x==row['name_x']].link
    years = places[places.name_x==row['name_x']].year
    
    popup_list = []
    
    for i in range(n_link):
        popup_list.append(html_str.format(links.iloc[i], years.iloc[i]))

    return name + str(popup_list).replace('[','').replace(']','').replace(',','').replace("'", '')
    
    



# add markers
feature_group = folium.FeatureGroup()

for index, row in places.iterrows():

    feature_group.add_child(folium.Marker(location=[row['lat'], row['lon']],
                                          
                                          popup=get_popup_links(),                                          
                                          
                                              icon=folium.Icon(color='darkblue'))
                           )
    

# add to map                                   
m.add_child(feature_group)



legend_html = '''
                    <div style="position: fixed; 
                             bottom: 90px; left: 50px; width: 120px; height: 90px; 
                             border:0px solid grey; z-index:9999; font-size:16px;
                             ">; <br>
                             <i class="fa fa-map-marker fa-2x" style="color:darkblue"> </i><br>
                               Click on the markers\nto display links to photos &nbsp; </i><br>
                            
                     </div>
                '''


m.get_root().html.add_child(folium.Element(legend_html))
m.save('HUGG-map-v1.html')
m